# 기본 벡터 검색
- 정렬된 순서를 기록한 인덱스 파일을 기반으로 하는 ANN 검색
- 수신된 검색 요청에 포함된 쿼리 벡터를 기반으로 벡터 임베딩 하위 집합을 찾아 쿼리 벡터와 하위 그룹에 있는 벡터를 비교하여 반환

참고자료: https://milvus.io/docs/ko/single-vector-search.md

## 개요
- kNN은 벡터 공간의 모든 벡터를 검색 요청에 포함된 쿼리 벡터와 비교하여 가장 유사한 벡터를 찾아내야 하므로 시간과 리소스가 많이 소요됨
- ANN은 벡터 임베딩의 정렬된 순서를 기록하는 인덱스 파일을 요청
- 검색 요청이 들어오면 인덱스 파일을 참조로 사용해 쿼리 벡터와 가장 유사한 벡터 임베딩이 포함된 하위 그룹을 빠르게 찾음
- 그 다음 지정된 메트릭 유형을 사용해 쿼리 벡터와 하위 그룹 벡터의 유사성을 측정하고, 쿼리 벡터와 유사성을 기준으로 그룹 구성원을 정렬하고, 상위 K개의 그룹 구성원을 파악할 수 잇음
- ANN 검색은 미리 구축된 인덱스에 따라 달라지며, 검색 처리량, 메모리 사용량 및 검색 정확도는 선택한 인덱스 유형에 따라 달라질 수 있음

## 단일 벡터 검색
- ANN 검색에서 단일 벡터 검색은 하나의 쿼리 벡터만 포함하는 검색을 의미
- 미리 구축된 인덱스와 검색 요청에 포함된 메트릭 유형에 따라 Milvus는 쿼리 벡터와 가장 유사한 상위 K개의 벡터를 찾음
- 이 섹션에서는 단일 벡터 검색을 수행함
- 검색 요청은 단일 쿼리 벡터를 전달하고 Milvus가 내적 곱(IP)을 사용하여 쿼리 벡터와 컬렉션의 벡터 간의 유사도를 계산하고 가장 유사한 세 개의 벡터를 반환하도록 요청

In [3]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

# 4. Single vector search
# vector 검색을 사용할 때 
query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]
res = client.search(
    collection_name="quick_setup",
    anns_field="vector",
    data=[query_vector],
    limit=3,
    search_params={"metric_type": "COSINE"} # metric type을 처음 DB 만들 때 해줘야 함
)

for hits in res:
    for hit in hits:
        print(hit)

{'id': 5, 'distance': 0.959449052810669, 'entity': {}}
{'id': 1, 'distance': 0.8689615726470947, 'entity': {}}
{'id': 7, 'distance': 0.8660879731178284, 'entity': {}}


## 대량 벡터 검색
- 검색 요청에 여러 개의 쿼리 벡터를 포함할 수 있음
- Milvus는 쿼리 벡터에 대해 ANN 검색을 병렬로 수행하여 두 세트의 결과를 반환

In [4]:
query_vectors = [
    [0.041732933, 0.013779674, -0.027564144, -0.013061441, 0.009748648],
    [0.0039737443, 0.003020432, -0.0006188639, 0.03913546, -0.00089768134]
]

res = client.search(
    collection_name="quick_setup",
    data=query_vectors,
    limit=3,
)

for hits in res:
    print("TopK results:")
    for hit in hits:
        print(hit)

TopK results:
{'id': 11, 'distance': 0.6200898289680481, 'entity': {}}
{'id': 1, 'distance': 0.49548205733299255, 'entity': {}}
{'id': 3, 'distance': 0.32014700770378113, 'entity': {}}
TopK results:
{'id': 14, 'distance': 0.7571712732315063, 'entity': {}}
{'id': 6, 'distance': 0.5678123235702515, 'entity': {}}
{'id': 13, 'distance': 0.5497628450393677, 'entity': {}}


## 파티션에서 ANN 검색
- 컬렉션에 여러 개의 파티션을 만들었고 검색 범위를 특정 수의 파티션으로 좁힐 수 있다고 가정
- 이 경우 검색 요청에 대상 파티션 이름을 포함시켜 검색 범위를 지정된 파티션 내로 제한할 수 있음
- 검색에 관련된 파티션 수를 줄이면 검색 성능이 향상

In [5]:
# 4. Single vector search
query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]
res = client.search(
    collection_name="quick_setup",
    partition_names=["partitionA"],
    data=[query_vector],
    limit=3,
)

for hits in res:
    print("TopK results:")
    for hit in hits:
        print(hit)

TopK results:
{'id': 13, 'distance': 0.6395301818847656, 'entity': {}}
{'id': 12, 'distance': 0.5408027768135071, 'entity': {}}
{'id': 11, 'distance': 0.475819855928421, 'entity': {}}


## 출력 필드 사용
- 검색 결과에서 Milvus는 기본적으로 상위 K 벡터 임베딩을 포함하는 엔티티의 기본 필드 값과 유사도 거리/점수를 포함
- 벡터 필드와 스칼라 필드를 포함한 대상 필드의 이름을 검색 요청에 출력 필드를 포함하면 검색 결과에 이러한 엔티티의 다른 필드 값이 포함되도록 할 수 있음

In [9]:
# 4. Single vector search
query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]

res = client.search(
    collection_name="quick_setup",
    data=[query_vector],
    limit=3, # The number of results to return
    search_params={"metric_type": "COSINE"},
    output_fields=["color"]
)

print(res)

data: [[{'id': 5, 'distance': 0.959449052810669, 'entity': {'color': 'black_9955'}}, {'id': 1, 'distance': 0.8689615726470947, 'entity': {'color': 'red_7319'}}, {'id': 7, 'distance': 0.8660879731178284, 'entity': {'color': 'white_5015'}}]]


## 제한 및 오프셋 사용
- 검색 요청에 포함된 limit 매개변수가 검색 결과에 포함할 엔티티의 수를 결정
- 단일 검색에서 반환할 최대 엔티티 수를 지정하며, top-K라고 함
- 페이지 매김 쿼리를 수행하려는 경우 루프를 사용하여 여러 개의 검색 요청을 보내고 각 쿼리 요청에 제한 및 오프셋 매개변수를 전달할 수 있음
- 구체적으로, 현재 쿼리 결과에 포함하려는 엔티티의 수로 Limit 매개변수를 설정하고, 이미 반환된 엔티티의 총 수로 Offset을 설정
- 단일 ANN 검색에서 limit와 offet의 합은 16,384 미만이어야 함

In [ ]:
query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]

res = client.search(
    collection_name="quick_setup",
    data=[query_vector],
    limit=3, # The number of results to return
    search_params={
        "metric_type": "COSINE", 
        # highlight-next-line
        "offset": 10 # The records to skip
    }
)